
1) Скачать нейросети ResNet и написать короткую процедуру для предсказания класса изображения

2) Скачать нейросеть BERT для лингвистических задач и реализовать процедуру классификации текстов (без оглядки на качество классификации)

### Задача 1: Классификация изображений с помощью ResNet

Для этой задачи будем использовать предобученную нейросеть ResNet из библиотеки torchvision.

In [1]:
# Импорт библиотек
import torch
from torchvision import models, transforms
from PIL import Image
import requests
import json

### Загрузка и инициализация предобученной модели ResNet

In [2]:
# Загрузка предобученной модели ResNet
resnet = models.resnet101(pretrained=True)
resnet.eval()  # Перевод модели в режим оценки

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 127MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Функция для предсказания класса изображения

In [ ]:
def predict_image_class(image_path, model, classes):
    # Преобразования для подготовки изображения
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Загрузка и преобразование изображения
    image = Image.open(image_path)
    image = preprocess(image).unsqueeze(0)

    # Предсказание класса изображения
    with torch.no_grad():
        outputs = model(image)
        _, predicted_class = outputs.max(1)
        class_id = predicted_class.item()

    # Возвращение названия класса
    return classes[class_id]

### Скачивание и загрузка меток классов ImageNet

In [ ]:
url = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
response = requests.get(url)
if response.status_code == 200:
    imagenet_classes = response.json()
else:
    raise ValueError(f"Не удалось загрузить классы ImageNet: статус {response.status_code}")

### Предсказание класса для изображения

In [ ]:
image_path = 'corgi.jpg'
predicted_class = predict_image_class(image_path, resnet, imagenet_classes)
print(f"Предсказанный класс изображения: {predicted_class}")

Предсказанный класс изображения: Pembroke Welsh Corgi


### Задача 2: Классификация текстов с помощью BERT

In [ ]:
# Импортируем необходимые библиотеки
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [ ]:
# Загрузка предобученной модели BERT и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Процедура для классификации текстов
def classify_text(text):
    # Токенизация текста
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")

    # Предсказание
    outputs = model(**inputs)

    # Предсказание класса
    predictions = torch.argmax(outputs.logits, dim=1)

    # Возвращаем класс текста
    return predictions[0].item()